# Machine Learning: Deep Learning

Material for the [2018 Asterics and Obelics School](https://indico.in2p3.fr/event/16864/): [Table of Contents](Contents.ipynb)

Content is maintained on [github](https://github.com/Asterics2020-Obelics/School2018/tree/master/machinelearning) and distributed under a [BSD3 license](https://opensource.org/licenses/BSD-3-Clause).

![sponsor-logos](img/sponsor-logos.png)

## Neural Network Architectures for Deep Learning

We previously took a bottom-up look at how a neural network is composed of basic building blocks. Now, we take a top-down look at some of the novel network architectures that are enabling the current [deep-learning revolution](https://www.nytimes.com/2016/12/14/magazine/the-great-ai-awakening.html):
 - Convolutional networks
 - Unsupervised learning networks
 - Recurrent networks
 - Reinforcement learning
 
We conclude with some reflections on where "deep learning" is headed.

### Convolutional Networks

A key benchmark problem in the ML community is identifying natural objects in photos. Since 2010, the community has run a classification competition using the [ImageNet](https://en.wikipedia.org/wiki/ImageNet) database of 14M web images labeled with 1000 categories (including 90 breeds of dog!)

The dramatic improvement (25% to 16% error rate) in 2012 is considered the start of the current deep learning "revolution", and used a convolutional neural network.

![imagenet](img/imagenet.png)

Industry applications have focused on classifying natural images (photos, video frames) and now exceed "human performance" on the standard benchmarks.

Most image processing tasks now start from a pre-trained highly complex state-of-the-art CNN, then make small perturbations to the weights using a new training set.

However, our scientific images often contain fundamentally different information so it seems unlikely that this is the optimal approach.

Convolution architectures are ideal for 1D data, not just 2D images, so CNNs have great potential for ML problems with 1D time series and spectroscopic datasets.

### Networks for Unsupervised Learning

Neural networks are usually used for supervised learning since their learning is accomplished by optimizing a loss function that compares the network's outputs with some target values.

However, it is possible to perform unsupervised learning if we can somehow use the same data for both the input values and the target output values.

This requires that the network have the same number of input and output nodes, and effectively means that we are asking it to learn the identify function, which does not sound obviously useful.

Suppose we have a single hidden layer with the same number of nodes as the input and output layers, then all the network has to do is pass each input value through to the output, which does not require any training at all!  

However, if the hidden layer has fewer nodes then we are asking the network to solve a more interesting problem: how can the input dataset be encoded and then decoded.

This is the same **dimensionality reduction** problem we discussed [earlier](Dimensionality.ipynb), and is known as an **autoencoder network** since it learns to encode itself.

![AutoEncoder architecture](img/AutoEncoder.png)

The network can be thought of as the combination of separate encoder and decoder networks, with the encoder feeding its output latent variables $\mathbf{z}$ into the decoder. Although the architecture looks symmetric, the encoder and decoder will generally learn different parameters because of the asymmetry introduced by nonlinear activations.

This is a high-level design pattern and the internal architectures of the encoder and decoder networks should be customized for the type of data being encoded (and typically combine convolutional and dense layers).

See this [notebook](https://github.com/dkirkby/MachineLearningStatistics/blob/master/notebooks/DeepLearning.ipynb) for an example based on processing 1D time series data.

### Variational Autoencoder

A further refinement on the autoencoder idea is to learn a posterior probability distribution in the latent variable space, instead of simply mapping each input to its corresponding point in the latent variable space. This is easier than it sounds if we assume that the posterior for each individual sample is described by an (uncorrelated) multi-variate Gaussian.

In practice, we simply need to learn how to transform each input to a corresponding vector of means $\mathbf{\mu}$ and sigmas $\mathbf{\sigma}$ in the latent variable space, effectively doubling the the number of output values for the encoder network, now re-interpreted as a posterior inference network.

Since this first stage is effectively a variational model of the posterior, learning its parameters is equivalent to performing a variational inference and we call this approach a **variational autoencoder (VAE)**.

The decoder network is also re-interpreted as a probabilistic model for generating realistic (smoothed) data. It is a generator rather than a decoder since it is no longer directly connected to the inputs. After training, it can be useful as a standalone simulator of realistic inputs.

Finally we need a prior we we take to be a unit (multivariate) Gaussian in the latent-variable space.  This is an arbitrary choice, but some choice is necessary in order to setup a Bayesian competition between new inputs and prior knowledge.

In effect, we are reversing the way we usually build a model, which is to specify the parameters then ask what their prior should be.  Instead, we are specifying the prior and then learning a (latent) parameter space that can explain the data with this prior.

![Variational autoencoder architecture](img/VariationalAutoEncoder.png)

See this [tutorial](https://arxiv.org/abs/1606.05908) for more details on the probabilistic background of VAE.

### Generative-Adversarial Network

Building on the theme of a probabilistic generator, we can set up an "arms race" between two networks:
 - A generative model that learns to synthesize realistic data.
 - An adversarial classifier that learns to distinguish between real and generated data.

This is the central idea of a **generative-adversarial network (GAN)**, which is a [recent idea](https://arxiv.org/abs/1406.2661) (2014):

![Generative adversarial network](img/GAN.png)

Each training step now has several parts:
 - Generate some random data.
 - Test how well the discriminator identifies the generated data as a fake.
 - Feed the same discriminator some real data.
 - Test how well the discriminator identifies the real data as real.

Optimizing the loss function then simultaneously improves the generator and the discriminator. The usual goal of training a GAN is to obtain a useful generator of realistic data.

See this [blog post](http://kvfrans.com/generative-adversial-networks-explained/) for an example based on image generation.

## Recurrent Networks

All the architectures we have seen so far are **feed-foward** networks, with input data always from left (input layer) to right (output layer). A **recurrent neural network (RNN)** adds links that feed back into a previous layer. This simple modification adds significant complexity but also expressive power (comparable to the electronics revolution associated with the idea of transistor feedback).

Architectures with feedback are still maturing but some useful building blocks have emerged, such as the [long short-term memory unit](https://en.wikipedia.org/wiki/Long_short-term_memory), which allows a network to remember some internal state but also sometimes forget it based on new input.

Some practical considerations for RNN designs:
 - The order of training data is now significant and defines a "model time", but the network can be reset whenever needed.
 - Input data can be packaged into variable-length messages that generate variable (and different) length output messages. This is exactly what language translation needs.
 - Optimization of the weights using gradients is still possible but requires "unrolling" the network by cloning it enough times to process the longest allowed messages.
 
A feed-foward network implements a **universal approximating function**. Since the internal state of an RNN acts like local variables, you can think of an RNN as a **universal approximating program**.

See this [blog post](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) for an example based on natural language synthesis.

## Reinforcement Learning

The architectures we have seen so far all have target output values associated with each input sample, which are necessary to update the network parameters during the learning (loss optimization) phase:

![Sample learning](img/SampleLearning.png)

However, we can relax this requirement of being able to calculate a loss after each new input as long as we eventually get some feedback on how well our input-to-output mapping is doing.  This is the key idea of **reinforcement learning (RL)**:

![Reinforcement learning](img/ReinforcementLearning.png)

A RL network watches some external "reality" (which is often simulated) and learns a policy for how to take actions.  A sequence of actions eventually leads to some feedback, which is then used to take a single step in optimizing the policy network's parameters:

![Policy network](img/PolicyNetwork.png)

See this [blog post](http://karpathy.github.io/2016/05/31/rl/) for an example based on image generation.

## Deep Learning Outlook

The depth of "deep learning" comes primarily from network architectures that stack many layers. In another sense, deep learning is very shallow since it often performs well using little to no specific knowledge about the problem it is solving, using generic building blocks.

The field of modern deep learning [started around 2012](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) with a CNN breakthrough in ImageNet, and when the necessary large-scale computing and datasets were available. Massive neural networks are now the state of the art for many benchmark problems, including image classification, speech recognition and language translation.

However, less than a decade into the field, there are signs that deep learning is reaching its limits. Some of the pioneers are focusing on new directions such as [capsule networks](https://arxiv.org/abs/1710.09829) and [causal inference](https://arxiv.org/abs/1801.04016). Others are taking a [critical look](https://arxiv.org/abs/1801.00631) at the current state of the field:
 - Deep learning does not use data efficiently.
 - Deep learning does not integrate prior knowledge.
 - Deep learning often give correct answers but without associated uncertainties.
 - Deep learning applications are hard to interpret and transfer to related problems.
 - Deep learning is excellent at learning stable input-output mappings but does cope well with varying conditions.
 - Deep learning cannot distinguish between correlation and causation.
 
These are mostly concerns for the future of neural networks as a general model for artificial intelligence, but they also limit the potential of scientific applications.

However, there are many challenges in scientific data analysis and interpretation that could benefit from deep learning approaches.

A practical outlook for deep learning in science:
 - Follow the current research, which is mostly driven by industry and computer scientists today.
 - Industry solutions are often a good starting point, so learn and use them.
 - Many scientific problems will likely benefit from fundamentally different approaches.
 - There are lots of opportunities for new ideas to have a big impact in science.